In [61]:
#Setup
import numpy as np
from numpy.testing import assert_almost_equal
import scipy.integrate as integrate
from scipy.stats import norm
import matplotlib.pyplot as plt 
rng = np.random.default_rng(seed = 42)

## Part 0 - Setup

In [64]:
### Constants
sigma1 = 0.11
sigma2 = 0.13
T = 1.7
mu = 0.13
r = 0.01
S0 = 149
time_break = 0.3
K = 188
KH = S0
n_sims = 1_000
n_timesteps = 1_000

In [63]:
### Helper Functions

### Time-varying vol

def time_varying_vol(t):
    if t > time_break:
        return sigma1 + sigma2 * (t - time_break)/(T - time_break)
    return sigma1

assert_almost_equal(np.array([time_varying_vol(0.2), time_varying_vol(0.4), time_varying_vol(T)]) ,
                    np.array([sigma1, 0.11928571428, sigma1 + sigma2]))

def integrated_vol(t):
    squared_vol = lambda x: time_varying_vol(x) ** 2
    result = integrate.quad(squared_vol, 0, t)
    return result[0]

assert_almost_equal(np.array([integrated_vol(0.3), integrated_vol(1), integrated_vol(1.7)]),
                    np.array([0.3 * 0.11 ** 2, 0.018090833333, 0.0484766666]))

### Black-Scholes

def black_scholes_call_pricing(S0,K,T,r,sigma):
    d1 = 1/(sigma * np.sqrt(T)) * (np.log(S0/K) + (r + sigma ** 2 / 2) * T)
    d2 = d1 - sigma * np.sqrt(T) 
    price =  S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    delta = norm.cdf(d1)
    gamma = norm.pdf(d1)/(S0 * sigma * np.sqrt(T))
    return price, delta, gamma

def black_scholes_put_pricing(S0,K,T,r,sigma):
    # note that C - P = S0 - K * exp(-r*T), put-call parity
    d1 = 1/(sigma * np.sqrt(T)) * (np.log(S0/K) + (r + sigma ** 2 / 2) * T)
    price =  black_scholes_call_price(S0,K,T,r,sigma) + K * np.exp(-r* T) - S0
    delta = norm.cdf(d1) - 1
    gamma = norm.pdf(d1)/(S0 * sigma * np.sqrt(T))
    return price, delta, gamma

np.testing.assert_almost_equal(np.array([black_scholes_call_pricing(100, 100, 1, 0.01, 0.2),black_scholes_put_pricing(100, 100, 1, 0.01, 0.2)]),
                                np.array([(8.43332, 0.55962, 0.01972),(7.43831, -0.44038, 0.01972)]) , decimal=5)


## Q1 - Delta Hedging

In [75]:
sigma = integrated_vol(T)

def simulate_gbm(S0, mu, sigma, T, n_steps, n_sims):
    
    paths = np.zeros([n_sims, n_steps+1])
    dt = T/n_steps
    paths[:, 0] = S0

    times = np.arange(0, T, dt) 

    norm_rvs = rng.normal(size = [n_sims, n_steps])

    for i in range(n_steps):
        paths[:, i + 1] = paths[:, i] * np.exp((mu - 0.5 * sigma ** 2)* dt + sigma * np.sqrt(dt) * norm_rvs[:, i])
                                               
    return paths, times

paths, times = simulate_gbm(S0, mu, sigma, T, n_timesteps, n_sims)



(array([[149.        , 148.97145216, 148.72141276, ..., 179.34414008,
         179.56983791, 179.21566386],
        [149.        , 149.24151128, 148.87022481, ..., 153.73416044,
         154.0052452 , 153.95289233],
        [149.        , 148.88018109, 149.33033055, ..., 202.32877582,
         202.7993257 , 202.81507574],
        ...,
        [149.        , 148.93084302, 148.86074953, ..., 188.5030567 ,
         188.51937544, 188.70021501],
        [149.        , 148.96753275, 149.14669883, ..., 182.99559574,
         183.09424502, 183.30483523],
        [149.        , 149.59004652, 149.74487714, ..., 180.83863261,
         180.98073699, 180.55709654]]),
 array([0.    , 0.0017, 0.0034, 0.0051, 0.0068, 0.0085, 0.0102, 0.0119,
        0.0136, 0.0153, 0.017 , 0.0187, 0.0204, 0.0221, 0.0238, 0.0255,
        0.0272, 0.0289, 0.0306, 0.0323, 0.034 , 0.0357, 0.0374, 0.0391,
        0.0408, 0.0425, 0.0442, 0.0459, 0.0476, 0.0493, 0.051 , 0.0527,
        0.0544, 0.0561, 0.0578, 0.0595, 0.0612, 0